# 🚀 LSTM Stock Prediction - Quick Start (Google Colab)

**One-click stock prediction with LSTM + Attention mechanism!**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fenago/lstm-attention-stock-prediction/blob/main/google_colab_quickstart.ipynb)

**Author:** Dr. Ernesto Lee | [drlee.io](https://drlee.io)  
**GitHub:** [fenago/lstm-attention-stock-prediction](https://github.com/fenago/lstm-attention-stock-prediction)

---

## What This Notebook Does

1. ✅ Installs all dependencies
2. ✅ Fetches stock data (default: AAPL)
3. ✅ Trains LSTM model with attention
4. ✅ Shows predictions vs actual prices
5. ✅ Predicts next 4 trading days
6. ✅ Downloads trained model

**Total Runtime:** ~5-10 minutes on Colab

---

## 🎯 Quick Start

Just click **Runtime → Run all** and wait!

Or run each cell step-by-step ⬇️

---

## Step 1: Install Dependencies

In [ ]:
%%capture
!pip install tensorflow keras yfinance numpy pandas matplotlib scikit-learn -q
print("✅ All packages installed!")

## Step 2: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, AdditiveAttention, Concatenate, Lambda
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

print(f"✅ TensorFlow {tf.__version__} ready!")

## Step 3: Configuration

**💡 Tip:** Change `TICKER` to predict any stock!

In [ ]:
#@title Configuration { run: "auto" }
TICKER = "AAPL" #@param {type:"string"}
START_DATE = "2020-01-01" #@param {type:"date"}
END_DATE = "2024-01-01" #@param {type:"date"}
SEQUENCE_LENGTH = 60 #@param {type:"slider", min:30, max:120, step:10}
EPOCHS = 50 #@param {type:"slider", min:20, max:100, step:10}

print(f"📊 Stock: {TICKER}")
print(f"📅 Date Range: {START_DATE} to {END_DATE}")
print(f"🔢 Sequence Length: {SEQUENCE_LENGTH} days")
print(f"🔄 Max Epochs: {EPOCHS}")

## Step 4: Fetch Data

In [ ]:
print(f"Fetching {TICKER} data...")
data = yf.download(TICKER, start=START_DATE, end=END_DATE)
data = data.fillna(method='ffill').fillna(method='bfill')

print(f"✅ Fetched {len(data)} trading days")
print(f"📈 Price range: ${data['Close'].min():.2f} - ${data['Close'].max():.2f}")

# Plot
plt.figure(figsize=(14, 5))
plt.plot(data.index, data['Close'], linewidth=2)
plt.title(f'{TICKER} Stock Price History', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

data.tail()

## Step 5: Prepare Data

In [ ]:
# Extract Close prices
close_prices = data['Close'].values.reshape(-1, 1)

# Split data (80/20)
split_idx = int(len(close_prices) * 0.8)
train_data = close_prices[:split_idx]
test_data = close_prices[split_idx:]

# Scale data (FIT on train only!)
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, SEQUENCE_LENGTH)

# For test, combine last sequence from train
combined = np.concatenate([train_scaled[-SEQUENCE_LENGTH:], test_scaled])
X_test, y_test = create_sequences(combined, SEQUENCE_LENGTH)

# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f"✅ Training data: {X_train.shape}")
print(f"✅ Testing data: {X_test.shape}")

## Step 6: Build Model with Attention

In [ ]:
# Build model with attention
inputs = Input(shape=(SEQUENCE_LENGTH, 1))

# LSTM layers
lstm1 = LSTM(64, return_sequences=True)(inputs)
lstm1 = Dropout(0.2)(lstm1)
lstm1 = BatchNormalization()(lstm1)

lstm2 = LSTM(32, return_sequences=True)(lstm1)
lstm2 = Dropout(0.2)(lstm2)
lstm2 = BatchNormalization()(lstm2)

# Attention
attention = AdditiveAttention()([lstm2, lstm2])
concat = Concatenate()([lstm2, attention])
pooled = Lambda(lambda x: tf.reduce_mean(x, axis=1))(concat)

# Output
dense = Dense(32, activation='relu')(pooled)
dense = Dropout(0.2)(dense)
outputs = Dense(1)(dense)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(f"✅ Model built: {model.count_params():,} parameters")
model.summary()

## Step 7: Train Model

**⏱️ This may take 3-5 minutes...**

In [ ]:
# Train
val_idx = int(len(X_train) * 0.8)
X_train_final = X_train[:val_idx]
y_train_final = y_train[:val_idx]
X_val = X_train[val_idx:]
y_val = y_train[val_idx:]

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
]

history = model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

## Step 8: Plot Training History

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 4))

ax[0].plot(history.history['loss'], label='Train')
ax[0].plot(history.history['val_loss'], label='Validation')
ax[0].set_title('Loss')
ax[0].set_xlabel('Epoch')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

ax[1].plot(history.history['mae'], label='Train')
ax[1].plot(history.history['val_mae'], label='Validation')
ax[1].set_title('MAE')
ax[1].set_xlabel('Epoch')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 9: Evaluate & Visualize

In [ ]:
# Predict
predictions_scaled = model.predict(X_test)

# Inverse transform
predictions = scaler.inverse_transform(predictions_scaled)
actuals = scaler.inverse_transform(y_test.reshape(-1, 1))

# Metrics
mae = mean_absolute_error(actuals, predictions)
rmse = np.sqrt(mean_squared_error(actuals, predictions))
r2 = r2_score(actuals, predictions)

print("📊 Test Set Performance:")
print(f"  MAE:  ${mae:.2f}")
print(f"  RMSE: ${rmse:.2f}")
print(f"  R²:   {r2:.4f}")

# Get test dates
test_dates = data.index[split_idx + SEQUENCE_LENGTH:]

# Plot
plt.figure(figsize=(16, 6))
plt.plot(data.index[-200:], data['Close'].values[-200:], label='Historical', alpha=0.7, linewidth=2)
plt.plot(test_dates, actuals, label='Actual (Test)', color='green', marker='o', markersize=3)
plt.plot(test_dates, predictions, label='Predicted (Test)', color='red', marker='x', markersize=3)
plt.axvline(x=test_dates[0], color='black', linestyle='--', label='Train/Test Split')
plt.title(f'{TICKER} Stock Prediction - LSTM with Attention', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Step 10: Predict Future (Next 4 Days)

In [ ]:
# Get last 60 days
last_sequence = close_prices[-SEQUENCE_LENGTH:]
last_sequence_scaled = scaler.transform(last_sequence)

# Predict next 4 days iteratively
future_predictions = []
current_sequence = last_sequence_scaled.copy()

for i in range(4):
    current_batch = current_sequence.reshape(1, SEQUENCE_LENGTH, 1)
    next_pred = model.predict(current_batch, verbose=0)
    future_predictions.append(next_pred[0, 0])
    current_sequence = np.append(current_sequence[1:], next_pred[0, 0])

# Inverse transform
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

last_price = close_prices[-1][0]
last_date = data.index[-1]

print(f"\n🔮 Future Predictions from {last_date.date()}:")
print(f"Last known price: ${last_price:.2f}\n")

for i, pred in enumerate(future_predictions, 1):
    change = pred[0] - last_price
    pct_change = (change / last_price) * 100
    direction = "⬆️" if change > 0 else "⬇️"
    print(f"Day {i}: ${pred[0]:.2f} ({change:+.2f}, {pct_change:+.2f}%) {direction}")
    last_price = pred[0]

## Step 11: Save & Download Model

In [ ]:
# Save model and scaler
model.save('lstm_model.h5')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✅ Model saved!")

# Download files (Colab only)
try:
    from google.colab import files
    files.download('lstm_model.h5')
    files.download('scaler.pkl')
    print("📥 Files downloaded to your computer!")
except:
    print("💾 Files saved in current directory")

---

## 🎉 Done!

You successfully:
- ✅ Trained an LSTM model with attention
- ✅ Evaluated on test data
- ✅ Made future predictions
- ✅ Downloaded the trained model

---

## Next Steps

1. **Try different stocks** - Change `TICKER` and rerun
2. **Advanced version** - Check out the full tutorial with 35 technical indicators
3. **Learn more** - Read the complete Medium article
4. **Star the repo** - [github.com/fenago/lstm-attention-stock-prediction](https://github.com/fenago/lstm-attention-stock-prediction)

---

## Resources

- 📖 [GitHub Repository](https://github.com/fenago/lstm-attention-stock-prediction)
- 📝 [Medium Article](https://drlee.io)
- 💻 [Advanced Tutorial Notebook](https://github.com/fenago/lstm-attention-stock-prediction/blob/main/basic_tutorial.ipynb)

---

## ⚠️ Disclaimer

**This is for educational purposes only.** Do not use for actual trading without proper risk management and professional financial advice.

---

**Created by Dr. Ernesto Lee | [drlee.io](https://drlee.io)**

If this helped you, please ⭐ the repo!